In [28]:
import pandas as pd
import numpy as np
import statistics as stats

In [25]:
def hectaresToCuerdas(hectares):
    """
    converts hectares to cuerdas, but also converts units with a /hectares denominator (e.g. trees per hectare)
    
    """
    cuerdas = hectares * 0.3930395625
    return(cuerdas)
    
print(hectaresToCuerdas(2750))

1080.858796875


In [21]:
species = np.array([[5,4,5,4],[25,13,10,12],[23,11,8,10],[200,125,125,125]])
speciesData = pd.DataFrame(species.T, columns = ['fullYieldYear', 'totalYearsProduction', 'declineStarts', 'harvestWeightPerCuerda'])
hectaresPerCuerda = unitsToHectares(1, 1)

speciesData['harvestWeightPerHectare'] = speciesData['harvestWeightPerCuerda'] / hectaresPerCuerda # calculate how many pounds
# per hectare according to these stats
speciesData

,fullYieldYear,totalYearsProduction,declineStarts,harvestWeightPerCuerda,harvestWeightPerHectare
0,5,25,23,200,508.854627
1,4,13,11,125,318.034142
2,5,10,8,125,318.034142
3,4,12,10,125,318.034142


In [21]:
class TreeTemplate:
    
    # the example below represents the borbón tree
    # not sure if I'll keep it all this way, but it's a good start
    
    
    def __init__(self, lsOfAges, _sowDensity = 0): # use self to declare namespace
        """
        
        Parameters
        ----------
        
        lsOfAges : list of int
            `lsOfAges` is a list of the ages of all of the trees, where each integer element stands for one tree of that type
            
        _sowDensity : int
            `_sowDensity` refers to the density of trees per space. The standard, expected unit for this initializer is trees-per-hectare
        
        
        """
        self.ages = lsOfAges # ls of ages of the [insert tree name] trees
        self.trees = len(lsOfAges) # how many [insert tree name] there are
        self.sowDensity = _sowDensity # sow density in trees/hectare
        
        self.firstHarvest = {'year': 4, 'proportion': 0.2} # year of first harvest and proportion of harvest until full
        self.fullHarvest = {'year': 5, 'proportion': 1.0}  # year of first harvest and proportion of harvest
        self.descentHarvest = {'year': 23, 'proportionDescent': 0.2} # year that production descends and annual proportion descent
        self.pruneHarvest = {'yearShift': -5, 'proportionAscent': 0.2} # these are placeholder values to remember to add member val
        self.death = {'year': 25} # year in which trees are expelled from dataset
        
        self.treeHarvestCap = 200 # units, in this case ?? lbs i think
        self.totalHarvest = 0 # units, in this case ???
        
        
    def addTreesAuto(self, number, ages = 0):
        """
        
        Parameters
        ----------
        self : class
            required to change-by-reference members of the class
            
        number: int
            the number of trees the user will add to the set
            
        ages : int
            the age of the trees (which translates to the element in the list)
            the user will add to the set. default value is 0 because most
            trees are planted/added as seeds/saplings, however the param
            is adjustable because it might be useful if adding new land with
            existing trees.
            
        """
        for i in range(len(number)):
            self.ages.append(ages) # all trees planted begin at age 0
            
    def addTreeSet(self, ls):
        """
        
        Parameters
        ----------
        self : class
            required to change-by-reference members of the class
            
        ls : list
            a list (of int) where each element represents a 'tree,' with
            the element's value representing that specific tree's age
            
        see also:
            makeTreesFromLand
            
        """
        for i,e in enumerate(ls):
            self.ages.append(e)
            
    # this may be a subfunciton in the 'addYears' function
    def oneYear(self):
        """
        
        This function takes this entire set of trees and adjusts the member values in the class to grow/change/produce accordingly.
        The function uses preset parameters for the specific tree type to guide the flow-control.
        
        """
        for treeIndex, treeAge in enumerate(self.ages):
            if (treeAge < self.firstHarvest['year']):
                #print("Age before: ", treeAge)
                #print("Harvest before: ", self.totalHarvest)
                
                product = self.treeHarvestCap * self.firstHarvest['proportion']
                self.totalHarvest += product
                self.ages[treeIndex] += 1 # assure to reference the list and not the copy
                
                #print("Age after: ", self.ages[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
                
            elif ((treeAge >= self.fullHarvest['year']) and (treeAge < self.descentHarvest['year'])):
                #print("Age before: ", treeAge)
                #print("Harvest before: ", self.totalHarvest)
                
                product = self.treeHarvestCap * self.fullHarvest['proportion']
                self.totalHarvest += product
                self.ages[treeIndex] += 1
                
                #print("Age after: ", self.ages[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
            elif ((treeAge >= self.descentHarvest['year']) and (treeAge) < self.death['year']):
                #print("Age before: ", treeAge)
                #print("Harvest before: ", self.totalHarvest)
                
                yearsIntoDescent = treeAge - self.descentHarvest['year']
                proportion = self.fullHarvest['proportion'] - (yearsIntoDescent * self.descentHarvest['proportionDescent'])
                product = self.treeHarvestCap * proportion
                self.totalHarvest += product
                self.ages[treeIndex] += 1
                
                #print("Age after: ", self.ages[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
            elif (treeAge == self.death['year']):
                #print("Tree dies now")
                #print("Harvest before: ", self.totalHarvest)
                continue
                #print("Age after: ", self.ages[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
            else:
                print("""The number: %d, list index: %d is out of range:
                a tree can not be less than 0 years old, and a tree of this type can not be more than
                %d years of age"""%(treeAge, treeIndex, self.death['year']))
                break
                   
        self.ages[:] = [age for age in self.ages if (age >= 25)] # call-by-reference overwrite of ls removing dead trees. 
        # assure this ^ is outside of the loop & assure the list references the full index with '[:]'
                
            
    def getHarvest(self):
        """
        
        return the total harvest
        
        """
        return(self.totalHarvest)
    
    def setHarvestZero(self):
        """
        
        You must set Harvest to zero after each iteration of oneYear if you want to keep track of annual production
        as opposed to total time production
        
        """
        self.totalHarvest = 0

In [29]:
class BourbonCuerda:
    
    # the example below represents the borbón tree
    # not sure if I'll keep it all this way, but it's a good start
    
    
    def __init__(self, _cuerdas, _initialAgeOfTrees, _sowDensity = hectaresToCuerdas(2750)): # use self to declare namespace
        """
        
        Parameters
        ----------
        
        lsOfAges : list of int
            `lsOfAges` is a list of the ages of, where each integer element stands for one tree of that type
            
        _sowDensity : int
            `_sowDensity` refers to the density of trees per space. The standard, expected unit for this initializer is trees-per-hectare
        
        
        """
        self.totalCuerdas = _cuerdas
        self.sowDensity = _sowDensity # sow density in trees/hectare
        self.totalTrees = _cuerdas * _sowDensity
        self.initialAgeOfTrees = _initialAgeOfTrees
        
        
        #self.numOfTrees = _cuerdas * _sowDensity # how many [insert tree name] there are
        self.trees = [initialAgeOfTrees for i in range(self.totalTrees)]
        
        self.averageAgeOfTrees = stats.mean(self.trees)
        
        self.firstHarvest = {'year': 4, 'proportion': 0.2} # year of first harvest and proportion of harvest until full
        self.fullHarvest = {'year': 5, 'proportion': 1.0}  # year of first harvest and proportion of harvest
        self.descentHarvest = {'year': 23, 'proportionDescent': 0.2} # year that production descends and annual proportion descent
        self.pruneHarvest = {'yearShift': -5, 'proportionAscent': 0.2} # these are placeholder values to remember to add member val
        self.death = {'year': 25} # year in which trees are expelled from dataset
        
        self.cuerdaHarvestCap = 200 # units, in this case lbs, per cuerda
        self.harvestPerTree = self.cuerdaHarvestCap / _sowDensity # pull initial sow density because the other will change
        # if plants are added or if others die
        self.totalHarvest = 0 # units, in this case ???
        
        
    def addTreesAuto(self, numTrees, ages = 0): # adds trees to existing cuerdas
        """
        
        Parameters
        ----------
        self : class
            required to change-by-reference members of the class
            
        number: int
            the number of trees the user will add to the set
            
        ages : int
            the age of the trees (which translates to the element in the list)
            the user will add to the set. default value is 0 because most
            trees are planted/added as seeds/saplings, however the param
            is adjustable because it might be useful if adding new land with
            existing trees.
            
        """
        for i in range(len(number)):
            self.trees.append(ages) # all trees planted begin at age 0
            
        self.totalTrees += numTrees
        self.sowDensity = self.totalTrees * cuerdas
        self.averageAgeOfTrees = stats.mean(self.trees)
            
    def addTreeSet(self, ls): # adds trees to existing cuerdas
        """
        
        Parameters
        ----------
        self : class
            required to change-by-reference members of the class
            
        ls : list
            a list (of int) where each element represents a 'tree,' with
            the element's value representing that specific tree's age
            
        see also:
            makeTreesFromLand
            
        """
        numTrees = len(ls)
        
        for i,e in enumerate(ls):
            self.trees.append(e)
            
        self.totalTrees += numTrees
        self.sowDensity = self.totalTrees * cuerdas
        self.averageAgeOfTrees = stats.mean(self.trees)
            
    # this may be a subfunciton in the 'addYears' function
    
    def addCuerdas(self, cuerdas, ageOfTrees, treesPerCuerda = 2750):
        # assign new variables to avoid confusion
        # if user wants to add land without trees, set treesPerCuerda to zero (note: other variables in the class
        # will reflect this change)
        oldSowDens = self.sowDensity
        oldCuerdas = self.totalCuerdas
        newCuerdas = self.totalCuerdas + cuerdas
        
        self.sowDensity = (treesPerCuerda * (cuerdas / newCuerdas)) + (oldSowDens * (oldCuerdas / newCuerdas))
        # calculate weighted avg and push new value to self.treesPerCuerda
        
        # acually update total cuerdas after to avoid confusion
        self.totalCuerdas += cuerdas
        
        # append the new trees to the
        numTrees = cuerdas * treesPerCuerda
        for i in range(len(numTrees)):
            self.trees.append(ageOfTrees)
            
        self.totalTrees += numTrees
        self.averageAgeOfTrees = stats.mean(self.trees)
        
        
    def oneYear(self):
        """
        
        This function takes this entire set of trees and adjusts the member values in the class to grow/change/produce accordingly.
        The function uses preset parameters for the specific tree type to guide the flow-control.
        
        """
        for treeIndex, treeAge in enumerate(self.trees):
            if (treeAge < self.firstHarvest['year']):
                #print("Age before: ", treeAge)
                #print("Harvest before: ", self.totalHarvest)
                
                product = self.harvestPerTree * self.firstHarvest['proportion']
                self.totalHarvest += product
                self.trees[treeIndex] += 1 # assure to reference the list and not the copy
                
                #print("Age after: ", self.trees[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
                
            elif ((treeAge >= self.fullHarvest['year']) and (treeAge < self.descentHarvest['year'])):
                #print("Age before: ", treeAge)
                #print("Harvest before: ", self.totalHarvest)
                
                product = self.harvestPerTree * self.fullHarvest['proportion']
                self.totalHarvest += product
                self.trees[treeIndex] += 1
                
                #print("Age after: ", self.trees[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
            elif ((treeAge >= self.descentHarvest['year']) and (treeAge) < self.death['year']):
                #print("Age before: ", treeAge)
                #print("Harvest before: ", self.totalHarvest)
                
                yearsIntoDescent = treeAge - self.descentHarvest['year']
                proportion = self.fullHarvest['proportion'] - (yearsIntoDescent * self.descentHarvest['proportionDescent'])
                product = self.harvestPerTree * proportion
                self.totalHarvest += product
                self.trees[treeIndex] += 1
                
                #print("Age after: ", self.trees[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
            elif (treeAge == self.death['year']):
                #print("Tree dies now")
                #print("Harvest before: ", self.totalHarvest)
                continue
                #print("Age after: ", self.trees[treeIndex])
                #print("Harvest after: ", self.totalHarvest)
                
            else:
                print("""The number: %d, list index: %d is out of range:
                a tree can not be less than 0 years old, and a tree of this type can not be more than
                %d years of age"""%(treeAge, treeIndex, self.death['year']))
                break
                   
        self.trees[:] = [age for age in self.trees if (age >= 25)] # call-by-reference overwrite of ls removing dead trees. 
        # assure this ^ is outside of the loop & assure the list references the full index with '[:]'
        self.totalTrees = len(self.trees)
        self.averageAgeOfTrees = stats.mean(self.trees)
        self.sowDensity = self.totalTrees / self.cuerdas
            
    def getHarvest(self):
        """
        
        return the total harvest
        
        """
        return(self.totalHarvest)
    
    def setHarvestZero(self):
        """
        
        You must set Harvest to zero after each iteration of oneYear if you want to keep track of annual production
        as opposed to total time production
        
        """
        self.totalHarvest = 0

In [51]:
def get_yn():
    loop = 1
    while loop:
        choice = input("Confirm ([Y]/N):")
        if choice in ['y', 'Y', '1', '']:
            return True
        elif choice in ['n', 'N', '0']:
            return False
        else:
            print("Invalid choice. Please try again!\n")


In [7]:
def findSowDensity():
    # do something
    sowDensity = 0

In [56]:
def promptUnits():
    print("Please select the units of measurement of the land in question:")
    print("\t(1) Cuerdas")
    print("\t(2) Hectares")
    print("\t(3) Square Kilometers")
    print("\t(4) Acres")
    print("\t(5) Square Miles")
    
    select = input("Enter a number between one and five: ")
    
    if (select == 1) or (select == '1') or (select == 'one'): # cuerdas to hectares
        cuerdas = input("Enter the # of cuerdas: ")
        hectares = unitsToHectares(1,cuerdas)
        
    elif (select == 2) or (select == '2') or (select == 'two'):
        hectares = input("Enter the # of hectares: ")
        
    elif (select == 3) or (select == '3') or (select == 'three'):
        kilos = input("Enter the # of square kilometers: ")
        hectares = unitsToHectares(3, kilos)
            
    elif (select == 4) or (select == '4') or (select == 'four'):
        acres = input("Enter the # of acres: ")
        hectares = unitsToHectares(4, acres)
                
    elif (select == 5) or (select == '5') or (select == 'five'):
        miles = input("enter the # of miles: ")
        hectares = unitsToHectares(5, miles)
        
    else:
        print("Invalid input. Try again.")
    
    return(hectares)

In [17]:
def unitsToHectares(tipo, units):
    
    """
    This function is utilized in user input function, but is also function-al within code.
    
    """
    if (tipo == 1) or (tipo == '1'): # cuerdas to hectares
        hectares = units * 0.3930395625
        #hectares * 0.393???
        
    elif (tipo == 2) or (tipo == '2'): # hectares to hectares
        hectares = units
        
    elif (tipo == 3) or (tipo == '3'): # sq km to hectares
        hectares = 100 * units
            
    elif (tipo == 4) or (tipo == '4'): # acres to hectares
        hectares = units / 2.471
                
    elif (tipo == 5) or (tipo == '5'): # sq miles to hectares
        hectares = units * 258.999
        
    return(hectares)

In [8]:
def makeTreesFromLand(hectares, sowDensity, age, proportion = 1.0):
    """
    
    Parameters
    ----------
    
    hectares : int or float
        the amount of land containing the tree type of the ls the user would like to create
        
    sowDensity : int or float
        the average amount of trees per hectare
        
    age : int
        the age of the trees you would like to add to the list
        
    proportion : float
        the proportion of the land that is filled with this type of trees. 
        
        For example, if I had 5 hectares with ~ 75% borbón and 25% catuaí trees, and I wanted to calculate
        the number of borbón, I would enter `0.75` for proportion
    
    
    """
    if type(age) != int:
        age = int(round(age, 0))
        
    totalTrees = (hectares * sowDensity) * proportion
    totalTrees = int(round(totalTrees, 0))
    
    treeAges = []
    
    for i in range(totalTrees):
        treeAges.append(age)
        
    return(treeAges)
        

In [9]:
import matplotlib.pyplot as plt

In [14]:
borbonAge = 7
land = 4.5 # hectares
sowDens = 2500
proportion = 0.7
borbonLs = makeTreesFromLand(land, sowDens, borbonAge, proportion= proportion)
len(borbonLs)

7875

In [15]:
borbon = TreeTemplate(borbonLs, _sowDensity=sowDens)

In [28]:
years = 25

for i in range(years):
    borbon.oneYear()
    product = borbon.getHarvest()
    borbon.setHarvestZero()
    
    # now append to list and/or plot

AttributeError: 'TreeTemplate' object has no attribute 'getHarvest'